In [1]:
from langchain_core.prompts import ChatPromptTemplate

import os
from dotenv import load_dotenv

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

True

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

In [3]:
prompt_val = prompt.invoke({"topic": "dog"})
print(prompt_val)

messages=[HumanMessage(content='Tell me an interesting fact about dog')]


In [6]:
print(prompt_val.to_messages()[0].content)

Tell me an interesting fact about dog


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
result = model.invoke(prompt_val)
result

AIMessage(content='Dogs have an incredibly powerful sense of smell, estimated to be between 10,000 to 100,000 times more sensitive than humans. This incredible sense of smell allows them to detect certain diseases in humans, such as cancer, diabetes, and seizures, often before any symptoms are apparent.', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 14, 'total_tokens': 73, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-81f696fa-d7f6-4f4b-af59-1cd4bafd180e-0')

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

'Dogs have an incredibly powerful sense of smell, estimated to be between 10,000 to 100,000 times more sensitive than humans. This incredible sense of smell allows them to detect certain diseases in humans, such as cancer, diabetes, and seizures, often before any symptoms are apparent.'

### Now let´s do this LCEL

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

basicchain = model | output_parser

In [10]:
basicchain.invoke("hello!")

'Hello! How can I assist you today?'

In [11]:
chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

"Dogs have a sense of time and can determine how much time has passed based on their owner's daily routine. This is why they often know when it's time for meals, walks, or when their owner is coming home from work."

### Retrieval Augmented Generation with LCEL

In [12]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [13]:
retriever.get_relevant_documents("What does the dog want to eat?")

c:\Users\Axioo Pongo\Documents\GitHub\Udemy-Advanced-LangChain\.lc\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]

In [14]:
retriever.invoke("What does the dog want to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]

In [15]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [16]:
from operator import itemgetter

retrieval_chain = (
    {
        "context": (lambda x: x["question"]) | retriever,
        # "question": lambda x: x["question"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [17]:
retrieval_chain.invoke({"question": "What does the dog like to eat?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'The dog likes to eat pizza.'

In [18]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
retrieval_chain.invoke("What does the dog like to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Pizza'